In [6]:
import os

from google import genai
from google.genai import types 

In [8]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key=GOOGLE_API_KEY)

# Schema Function Declaration

In [9]:
CREATE_LEAD_FUNCTION = {
    "name": "create_lead",
    "description": "Tạo lead mới trong hệ thống CRM",
    "parameters": {
        "type": "object",
        "properties": {
            "lead_name": {
                "type": "string",
                "description": "Tên đầy đủ của lead"
            },
            "email": {
                "type": "string",
                "description": "Email liên hệ"
            },
            "phone": {
                "type": "string",
                "description": "Số điện thoại liên hệ"
            },
            "company": {
                "type": "string",
                "description": "Tên công ty"
            }
        },
        "required": ["lead_name", "email"]
    }
}

# Configure tools

In [16]:
tools = types.Tool(function_declarations=[CREATE_LEAD_FUNCTION])
config = types.GenerateContentConfig(
    tools=[tools],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(
        disable=False,
    ),
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(
            # max_tokens=1024,
            # temperature=0.7,
            mode='ANY'
        )
    )
)

# Prompt

In [12]:
contents = [
    types.Content(
        role="user",
        parts=[
            types.Part(
                text="Create a new Lead with name Alex Dane, email alex.dane@example.com, phone 123456789, company Example Corp"
            )
        ]
    )
]

In [13]:
response = client.models.generate_content(
    model="gemini-1.5-flash",
    contents=contents,
    config=config
)

# Function call

In [14]:
part = response.candidates[0].content.parts[0]
part

Part(
  function_call=FunctionCall(
    args={
      'company': 'Example Corp',
      'email': 'alex.dane@example.com',
      'lead_name': 'Alex Dane',
      'phone': '123456789'
    },
    name='create_lead'
  )
)

In [15]:
if hasattr(part, 'function_call') and part.function_call:
    fc_name = part.function_call.name
    args = part.function_call.args

    print(f"Function call detected: {fc_name} with arguments: {args}")

    # Mock API
    def create_lead(lead_name: str, email: str, phone: str = None, company: str = None):
        return {
            "status": "success",
            "lead": {
                "name": lead_name,
                "email": email,
                "phone": phone,
                "company": company
            }
        }
    
    result = create_lead(**args)
    print(f"API call result: {result}")

    function_response_part = types.Part.from_function_response(
        name=fc_name,
        response={"result": result}
    )

    final_content = [
        response.candidates[0].content,
        types.Content(
            role="user",
            parts=[function_response_part]
        )
    ]

    final_response = client.models.generate_content(
        model="gemini-1.5-flash",
        config=config,
        contents=final_content
    )

    print(f"Final response: {final_response.text}")
else:
    print("No function call detected.")

Function call detected: create_lead with arguments: {'company': 'Example Corp', 'phone': '123456789', 'lead_name': 'Alex Dane', 'email': 'alex.dane@example.com'}
API call result: {'status': 'success', 'lead': {'name': 'Alex Dane', 'email': 'alex.dane@example.com', 'phone': '123456789', 'company': 'Example Corp'}}
Final response: OK. I've created a lead named Alex Dane from Example Corp with the email address alex.dane@example.com and phone number 123456789. Is there anything else I can help with?

